In [1]:
import pandas as pd

In [2]:
_data = pd.read_csv('../resources/data.csv', skiprows=[2])

In [3]:
ans_key = _data.head(1)

In [4]:
data = _data.drop(_data.index[0])

## Adding more columns
* Test grade
* Answered questions

In [5]:
data["TestGrade"]=[None]*data.shape[0]
data["QuestionsAnswered"]=[None]*data.shape[0]

In [6]:
data.iloc[0] # Check to see if it work

Participant Code        22
Term                     1
Test                  Post
Instructor               A
LetterGrade              D
PercentGrade         54.83
Q1                       D
Q2                       C
Q3                       B
Q4                       C
Q5                       B
Q6                       A
Q7                       C
Q8                       B
Q9                       C
Q10                      A
Q11                      C
Q12                      B
Q13                      B
Q14                      C
Q15                      B
Q16                      D
Q17                      C
Q18                      C
Q19                      A
Q20                      C
Q21                      A
Q22                      C
Q23                      A
Q24                      B
Q25                      A
Q26                      B
Q27                      A
TestGrade             None
QuestionsAnswered     None
Name: 1, dtype: object

In [7]:
for index, row in data.iterrows(): print(index)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147


In [8]:
for x in range(27): print(data.iloc[0][x+5] == ans_key.iloc[0][x])

False
False
False
False
False
False
False
False
False
False
True
False
False
False
False
False
True
False
False
False
False
False
True
False
False
False
False


In [9]:
for index, row in data.iterrows():
    grade = 0
    answered = 0
    for x in range(27):
        if(not row.isna()[x+5]):
            answered += 1
            if(row[x+5] == ans_key.iloc[0][x]):
                grade += 1
    data.loc[index,"QuestionsAnswered"] = answered
    data.loc[index,"TestGrade"] = grade / 27.0

In [10]:
data.head()

,Participant Code,Term,Test,Instructor,LetterGrade,PercentGrade,Q1,Q2,Q3,Q4,...,Q20,Q21,Q22,Q23,Q24,Q25,Q26,Q27,TestGrade,QuestionsAnswered
1,22,1.0,Post,A,D,54.83,D,C,B,C,...,C,A,C,A,B,A,B,A,0.111111,27
2,50,1.0,Pre,A,F,12.41,A,NaN,A,NaN,...,B,A,NaN,C,NaN,NaN,NaN,NaN,0.111111,16
3,82,1.0,Pre,A,B+,84.94,C,C,A,C,...,B,A,D,C,A,E,B,B,0,26
4,82,1.0,Post,A,B+,84.94,B,D,D,D,...,B,C,C,A,NaN,D,D,C,0.148148,25
5,22,1.0,Pre,A,D,54.83,E,D,C,B,...,D,A,E,A,B,D,D,A,0.222222,27
